In [14]:
from qiskit import QuantumCircuit, execute
from qiskit.circuit import ParameterVector
import random
from qiskit import Aer
from qiskit.aqua.components.optimizers import ADAM
import numpy as np

Prepare 4 random 4-qubit quantum states of your choice.
Create and train a variational circuit that transforms input states into predefined output states. Namely
if random state 1 is provided, it returns state |0011>
if random state 2 is provided, it returns state |0101>
if random state 3 is provided, it returns state |1010>
if random state 4 is provided, it returns state |1100>
What would happen if you provided a different state?

Analyze and discuss the results.

In [28]:
def generate_random_circuit(random_state):
    qc = QuantumCircuit(4)
    if(random_state == 1):
        qc.x(random.randint(0,3))
        qc.h(random.randint(0,3))
    if(random_state == 2):
        qc.x(random.randint(0,3))
        qc.x(random.randint(0,3))
    if(random_state == 3):
        qc.h(random.randint(0,3))
        qc.h(random.randint(0,3))
    return qc


def get_prob_distribution(circuit,backend, params):
    #Assign parameters using the assign_parameters method
    bound_circuit = circuit.bind_parameters(params)
    bound_circuit.measure_all()
    raw_distrubution = execute(bound_circuit,backend,shots=1000).result().get_counts()
    for key in raw_distrubution.keys():
        raw_distrubution[key] = raw_distrubution[key]/1000
    return raw_distrubution

def cost_function(expected_vals,distribution,epsilon=0.0005):
    value = 0
    for label in expected_vals:
        if(label in distribution):
            value += np.log(max(epsilon,distribution[label]))
        else:
            value += np.log(epsilon)
    return (-1/len(expected_vals))*(value)


def objective_function(self, dataPoints, expected_classes, params):
    expects = list()
    for point in dataPoints:
        expects.append(self.get_expectation(point, params))
    return self.get_cost_mse(expects, expected_classes)

def train(X, Y, init_params):
    optimizer = ADAM(maxiter=100, )
    objective_function = lambda params: objective_function(X, Y, params)
    opt_params, value, _ = optimizer.optimize(len(init_params), objective_function, initial_point=init_params)
    return opt_params

def generate_anzats(n_qubits,n_layers,type_r=None):

    #ParameterVectors are initialized with a string identifier and an integer specifying the vector length
    parameters = ParameterVector('θ', (n_qubits*(n_layers+1)))
    circuit = QuantumCircuit(n_qubits)

    for layer in range(n_layers):
        for i in range(n_qubits):
            if(type_r == "qgan"):
                circuit.ry(parameters[n_qubits*layer+i], i)
            else:
                circuit.rx(parameters[n_qubits*layer+i], i)

        #Appending the entangling Controlled gates
        for i in range(n_qubits):
            if(type_r == "qgan"):
                circuit.cz(i,(i+1)%n_qubits)
            else:
                for j in range(i+1,n_qubits):
                    circuit.cx(i,j)

    #Appending one additional layer of parameterized Ry gates
    for i in range(n_qubits):
        if(type_r == "qgan"):
            circuit.ry(parameters[n_qubits*(n_layers)+i], i)
        else:
            circuit.rx(parameters[n_qubits*(n_layers)+i], i)
    circuit.barrier()
    return circuit


In [30]:
random_state = random.randint(1,4)
circuit = generate_random_circuit(random_state)
n = 4
num_layers = 2
circuit.append(generate_anzats(n,num_layers),[0,1,2,3])
param_list = [random.randint(0,20) for x in range(0,(n*(num_layers+1)))]
distr = get_prob_distribution(circuit,Aer.get_backend('qasm_simulator'), param_list, )
cost_function(['0001'],dict({'0001':1}))

-0.0